# Part 9 - এনক্রিপ্টড প্রোগ্রামগুলির পরিচয় (Intro to Encrypted Programs)

বিশ্বাস করুন বা না করুন, এনক্রিপ্ট করা ডেটা দিয়ে গণনা করা সম্ভব। অন্য কথায়, একটি প্রোগ্রাম চালানো সম্ভব যেখানে প্রোগ্রামের **সমস্ত ভেরিয়েবল** **এনক্রিপ্টেড** ( _All of the variables in the program are encrypted_ )

এই টিউটোরিয়ালে, আমরা এনক্রিপ্ট করা গণনার খুব প্রাথমিক সরঞ্জামগুলির মধ্য দিয়ে চলতে যাচ্ছি। বিশেষত, আমরা সিকিওর মাল্টি-পার্টির গণনা নামে পরিচিত একটি জনপ্রিয় পদ্ধতির উপর ফোকাস করতে যাচ্ছি। এই পাঠে, আমরা কীভাবে একটি এনক্রিপ্ট করা ক্যালকুলেটর তৈরি করতে শিখব যা এনক্রিপ্ট হওয়া সংখ্যায় গণনা সম্পাদন করতে পারে।

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

References: 
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)

# Step 1: সুরক্ষিত মাল্টি-পার্টির গণনা ব্যবহার করে এনক্রিপশন (Encryption Using Secure Multi-Party Computation)

এসএমপিসি(SMPC) প্রথম নজরে "এনক্রিপশন" এর চেয়ে বরং এক অদ্ভুত রূপ। কোনও ভেরিয়েবল এনক্রিপ্ট করার জন্য পাবলিক / প্রাইভেট কী ব্যবহার করার পরিবর্তে প্রতিটি মান একাধিকতে বিভক্ত হয় shares, যার প্রতিটি একটি ব্যক্তিগত কী হিসাবে চালিত হয়। সাধারণত, এই shares 2 বা ততোধিক মালিকদের মাঝে বিতরণ করা হবে। সুতরাং, ভেরিয়েবলটি ডিক্রিপ্ট(decrypt the variable) করার জন্য, সমস্ত মালিকদের ডিক্রিপশন অনুমোদনের জন্য সম্মত হতে হবে। সংক্ষেপে, প্রত্যেকের একটি ব্যক্তিগত চাবি আছে।

### এনক্রিপ্ট Encrypt()

সুতরাং, ধরা যাক আমরা একটি ভেরিয়েবল `x`" এনক্রিপ্ট "করতে চেয়েছিলাম, আমরা নিম্নলিখিত পদ্ধতিতে এটি করতে পারি।

 > এনক্রিপশন ফ্লোটস বা আসল সংখ্যা ব্যবহার করে না তবে এটি গাণিতিক জায়গায় ঘটে [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) যা মূলত এর মধ্যে পূর্ণসংখ্যা `0` এবং `Q-1`, যেখানে `Q` প্রধান এবং "যথেষ্ট বড়" যাতে স্থানটি আমাদের পরীক্ষাগুলিতে আমরা যে সমস্ত সংখ্যা ব্যবহার করি তা ধারণ করতে পারে। অনুশীলনে, একটি মান দেওয়া হয় `x` পূর্ণসংখ্যা, আমরা করি `x % Q` রিং এর মধ্যে ফিট। (এজন্যই আমরা সংখ্যা ব্যবহার করা এড়িয়ে চলি `x' > Q`).

In [1]:
Q = 1234567891011

In [2]:
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(890804432397, -2305631655, 346069090294)

আপনি এখানে দেখতে পাচ্ছেন, আমরা আমাদের পরিবর্তনশীল `x` কে 3 টি আলাদা শেয়ারে বিভক্ত করেছি, যা 3 টি পৃথক মালিককে পাঠানো যেতে পারে।

### ডিক্রিপ্ট Decrypt()

আমরা যদি এই 3 টি শেয়ার ডিক্রিপ্ট করতে চাইতাম, তবে আমরা কেবল তাদের একসাথে যোগ করতে পারি এবং ফলাফলের মডুলাস নিতে পারি (Mod Q)

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)

In [7]:
decrypt(a, b, c)

25

গুরুত্বপূর্ণভাবে লক্ষ করুন যে আমরা যদি মাত্র দুটি শেয়ার দিয়ে ডিক্রিপ্ট করার চেষ্টা করি তবে ডিক্রিপশনটি কার্যকর হয় না!

In [8]:
decrypt(a, b)

778460474681

সুতরাং, মানটি ডিক্রিপ্ট করার জন্য আমাদের সকল মালিকদের অংশ নেওয়া উচিত। এটি এই ভাবেই `shares` ব্যক্তিগত কীগুলির মতো কাজ করুন, মানটি ডিক্রিপ্ট করার জন্য অবশ্যই এগুলির সমস্ত উপস্থিত থাকতে হবে।

# Step 2: বেসিক গাণিতিক এসএমপিসি ব্যবহার করে (Basic Arithmetic Using SMPC)

তবে সিকিউর মাল্টি পার্টির গণনার সত্যিকারের অসাধারণ সম্পত্তি হ'ল গণনা সম্পাদন করার ক্ষমতা **যখন ভেরিয়েবলগুলি এখনও এনক্রিপ্ট করা থাকে**। নীচে সহজ সংযোজন প্রদর্শিত যাক।

In [9]:
x = encrypt(25)
y = encrypt(5)

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

### সফলতা (Success!!!)

এবং সেখানে আপনি এটা আছে! যদি প্রতিটি কর্মী (পৃথকভাবে) তাদের শেয়ারগুলি একসাথে যোগ করে, তবে ফলস্বরূপ শেয়ারগুলি সঠিক মানটিতে ডিক্রিপ্ট হবে (25 + 5 == 30). 

দেখা যাচ্ছে যে এসএমপিসি(SMPC) প্রোটোকল রয়েছে যা নিম্নলিখিত ক্রিয়াকলাপগুলির জন্য এই এনক্রিপ্ট করা গণনার অনুমতি দিতে পারে:

- যোগ (যা আমরা সবেমাত্র দেখেছি)
- গুণ
- তুলনা

এবং এই প্রাথমিক অন্তর্নিহিত আদিম ব্যবহার করে, আমরা নির্বিচারে গণনা সম্পাদন করতে পারি !!!

পরবর্তী বিভাগে, আমরা কীভাবে এই অপারেশনগুলি সম্পাদন করতে পাই সাইফ্ট গ্রন্থাগারটি ব্যবহার করতে পারি তা শিখতে চলেছি!

# Step 3: এসএমপিসি পাইসাইফ্ট ব্যবহার করে (SMPC Using PySyft)

পূর্ববর্তী বিভাগগুলিতে, আমরা এসএমপিসি-এর আশেপাশে কিছু বেসিক অন্তর্দৃষ্টিগুলি রূপরেখার কাজ করেছি বলে ধারণা করা হচ্ছে। যাইহোক, বাস্তবে আমাদের এনক্রিপ্ট করা প্রোগ্রামগুলি লেখার সময় আমরা আদিম ক্রিয়াকলাপগুলির সমস্ত নিজের হাতে লিখতে চাই না। সুতরাং, এই বিভাগে আমরা পাইসাইফ্ট ব্যবহার করে এনক্রিপ্ট করা গণনা কীভাবে করতে হবে তার মূল বিষয়গুলি নিয়ে যেতে পারি। বিশেষত, আমরা পূর্বে উল্লিখিত 3 আদিমগুলি কীভাবে করব তার উপর ফোকাস করতে যাচ্ছি: সংযোজন, গুণ এবং তুলনা।

প্রথমত, আমাদের কয়েকটি ভার্চুয়াল ওয়ার্কার তৈরি করতে হবে (যা আশা করি আপনি এখন আমাদের আগের টিউটোরিয়ালগুলির সাথে পরিচিত হন)।

In [12]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### সাধারণ এনক্রিপশন / ডিক্রিপশন (Basic Encryption/Decryption)

এনক্রিপশনটি(Encryption) কোনও পাইসাইফ্ট টেনসর নেওয়া এবং কল করার মতোই সহজ.share(). ডিক্রিপশন কল করার মতোই সহজ .get() ভাগ করা পরিবর্তনশীল উপর

In [13]:
x = torch.tensor([25])

In [14]:
x

tensor([25])

In [15]:
encrypted_x = x.share(bob, alice, bill)

In [16]:
encrypted_x.get()

tensor([25])

### এনক্রিপ্ট করা মানগুলিকে আত্মপ্রকাশ করে (Introspecting the Encrypted Values)

আমরা যদি Bob, Alice এবং Bill এর কর্মীদের কাছাকাছি তাকান, আমরা যে শেয়ারগুলি তৈরি হবে তা দেখতে পাব!

In [17]:
bob._objects

{}

In [18]:
x = torch.tensor([25]).share(bob, alice, bill)

In [19]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([3212861001891376707])

In [20]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([61371170032936135])

In [21]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([1337453846503075087])

এবং যদি আমরা চাইতাম, আমরা আগে যে বিষয়গুলি নিয়ে আলোচনা করেছি সেগুলি ব্যবহার করে আমরা এই মানগুলি ডিক্রিপ্ট করতে পারি !!!

In [22]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

যেমন আপনি দেখতে পাচ্ছেন, আমরা যখন ফোন করেছি`.share()` এটি কেবলমাত্র 3 টি শেয়ারে মূল্য বিভক্ত করে এবং প্রতিটি দলের কাছে একটি ভাগ প্রেরণ করে!

# এনক্রিপ্ট করা পাটীগণিত (Encrypted Arithmetic)

এবং এখন আপনি দেখতে পাচ্ছেন যে আমরা অন্তর্নিহিত মানগুলিতে গাণিতিক সম্পাদন করতে পারি! এপিআইটি(API) এমনভাবে তৈরি করা হয়েছে যাতে আমরা পাইটর্চ টেনসোর(PyTorch tensors) মতো সাধারণভাবে পাটিগণিত সম্পাদন করতে পারি।

In [23]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [24]:
z = x + y
z.get()

tensor([30])

In [25]:
z = x - y
z.get()

tensor([20])

# এনক্রিপ্ট করা গুণ (Encrypted Multiplication)

গুণনের জন্য আমাদের একটি অতিরিক্ত দল দরকার যারা ধারাবাহিকভাবে এলোমেলো সংখ্যা তৈরি করার জন্য দায়ী (এবং অন্য দলের কোনওটির সাথে মিলিত না হয়ে)। আমরা এই ব্যক্তিকে "ক্রিপ্টো সরবরাহকারী/crypto provide" বলি। সমস্ত নিবিড় উদ্দেশ্যে, ক্রিপ্টো সরবরাহকারী কেবল একটি অতিরিক্ত ভার্চুয়াল ওয়ার্কার, তবে এটি স্বীকার করা জরুরী যে ক্রিপ্টো সরবরাহকারী কোনও "মালিক" নন যে তার নিজের অংশীদারি নেই, তবে এমন কেউ আছেন যাকে একত্রিত করার জন্য বিশ্বাস করা উচিত নয় বিদ্যমান যে কোনও শেয়ারহোল্ডারের(shareholders) সাথে।

In [26]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [27]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [28]:
# multiplication

z = x * y
z.get()

tensor([125])

আপনি ম্যাট্রিক্স গুণও করতে পারেন

In [29]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [30]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

# এনক্রিপ্ট করা তুলনা (Encrypted comparison)
ব্যক্তিগত মূল্যগুলির মধ্যে ব্যক্তিগত তুলনা করাও সম্ভব। আমরা এখানে সিকিওরএনএন(SecureNN) প্রোটোকলে নির্ভর করি, যার বিবরণ পাওয়া যাবে [এখানে](https://eprint.iacr.org/2018/442.pdf). তুলনার ফলাফলটি একটি ব্যক্তিগত শেয়ার্ড টেনসরও।

In [31]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [32]:
z = x > y
z.get()

tensor([1])

In [33]:
z = x <= y
z.get()

tensor([0])

In [34]:
z = x == y
z.get()

tensor([0])

In [35]:
z = x == y + 20
z.get()

tensor([1])

আপনি max অপারেশনও করতে পারেন  

In [36]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [37]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

# অভিনন্দন সম্প্রদায় যোগদানের সময়! (Congratulations!!! - Time to Join the Community!)

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন। আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, AI এবং AI সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন।

### গিটহাবে পাইসিফ্ট কে স্টার দিন (Star PySyft on GitHub)

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হল রিপোসিটোরি গুলোতে ষ্টার করা
 এটি আমরা যে অসাধারণ সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাক যোগ দিন (Join our Slack!)

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হ'ল আমাদের সম্প্রদায়ে যোগদান করা। আপনি ফর্মটি পূরণ করে এটি করতে পারেন [http://slack.openmined.org](http://slack.openmined.org)

### একটি কোড প্রকল্পে যোগদান করুন! (Join a Code Project!)

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হল কোড অবদানকারী হয়ে উঠুন! যে কোনও সময় আপনি পাইসাইফ্ট গিটহাবে ইস্যু পৃষ্ঠাতে যেতে পারেন এবং "প্রকল্পগুলি" এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষ স্তরের সমস্ত টিকিট দেখিয়ে দেবে যে আপনি কোন প্রকল্পগুলিতে যোগদান করতে পারেন তার একটি ওভারভিউ দেয়! আপনি যদি কোনও প্রকল্পে যোগ দিতে না চান তবে আপনি কিছুটা কোডিং করতে চান তবে আপনি "ভাল প্রথম ইস্যু" চিহ্নিত গিথুব ইস্যুগুলি অনুসন্ধান করে আরও "ওয়ান অফ" মিনি-প্রকল্পগুলির সন্ধান করতে পারেন।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন (Donate)

আপনার যদি আমাদের কোডবেসে অবদান রাখার সময় না থাকে তবে তবুও সমর্থন দিতে চান, আপনি আমাদের ওপেন কালেক্টিভেরও Backer হয়ে উঠতে পারেন। সমস্ত অনুদান আমাদের ওয়েব হোস্টিং এবং অন্যান্য সম্প্রদায় ব্যয় যেমন হ্যাকাথনস এবং মেটআপগুলির (hackathons and meetups!) দিকে যায়।

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)